# Apeworx Setup

In [ ]:
from ape import accounts, chain, Contract, networks, project
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
networks.parse_network_choice('ethereum:local:foundry').__enter__()

In [ ]:
project_root = Path.cwd().parent

# Initialize Project Compilation
project.contracts
hyperdrive_user = accounts.test_accounts.generate_test_account()

# Give user 1 eth balance
hyperdrive_user.balance += int(1e18)

# Instantiate Contract Instances
base_ERC20 = project.IERC20Mint.at("0x5fbdb2315678afecb367f032d93f642f64180aa3")
hyperdrive = project.IHyperdrive.at("0x9fe46736679d2d9a65f0992f2272de9f3c7fa6e0")

# get current block
current_block = chain.blocks[-1].number
print(current_block)
pool_info = [hyperdrive.getPoolInfo().__dict__]
pool_info[0]["block_number_"] = current_block

def open_longs():
    # with notation avoids passing in sender each time
    with accounts.use_sender(hyperdrive_user):
        # Mint 1 million DAI
        base_ERC20.mint(int(1e24))

        # Approve ERC20 usage by contract
        base_ERC20.approve(hyperdrive, int(1e24))

        pool_info = hyperdrive.getPoolInfo().__dict__

        # Open Long on 10,000 DAI
        tx_receipt = hyperdrive.openLong(int(1e21), int(1e21), hyperdrive_user, False)
        pt_balance = tx_receipt.return_value
        
        print("user input %s base" %(int(1e21)))
        print("user received %s PTs" %(pt_balance))
        
        current_block = chain.blocks[-1]
        pool_info["block_number_"] = current_block.number

        return pool_info


for x in range(50):        
    pool_info += [open_longs()]



In [ ]:
df = pd.DataFrame(pool_info)

plt.figure(1)
plt.plot(df['block_number_'], df['shareReserves_'], label='Share Reserves')
plt.plot(df['block_number_'], df['bondReserves_'], label='Bond Reserves')
plt.plot(df['block_number_'], df['lpTotalSupply'], label='LP Supply')
plt.xlabel('Block Number')
plt.ylabel('Reserves & Supply')
plt.title('Local Longs Opened Consecutively')
plt.legend()
plt.show()
